In [1]:
from pyspark.sql.functions import udf, size, avg, count, col,sum, explode
import json
import datetime
env = "production"
env = "staging"

s3_bucket = "s3://mist-secorapp-{env}/mobile-client-marvis/mobile-client-marvis-{env}/".format(env=env)

date_day= datetime.datetime.today().strftime("%Y-%m-%d")
hr = '*'

date_day = "2020-05-1*"
# hr = '20'

s3_bucket += "dt={date}/{hr}/*.seq".format(date=date_day, hr=hr)

s3_path = s3_bucket 
print(s3_path)



# user_org_id = "38a18d4d-1623-4985-86d0-1bb06e5e2a48"  # UPS
# user_site_USNYELE = '9aa6ecdb-ddee-41b4-a8d9-872d7962b3c1'  #UPS USNYELE

# # user_org_id = '38a18d4d-1623-4985-86d0-1bb06e5e2a48'  # UPS
# user_site_USFLLON = 'c23667f3-47e9-44aa-9761-de49d34ed9f9' # USFLLON


s3://mist-secorapp-staging/mobile-client-marvis/mobile-client-marvis-staging/dt=2020-05-1*/*/*.seq


In [2]:

rdd = spark.sparkContext.sequenceFile(s3_path)


In [3]:
rdd.count()

4008

In [4]:
test= rdd.first()
test

(591,
 bytearray(b'{"Topic":"mobile-client-marvis-staging","HashKey":"3093b881-b5a8-4c3e-bd44-7464b0712a5d","Type":"Test send Basic Client Info plus Wifi or Cellular","Result":{"ClientInformation":{"Device":{"Manufacturer":"Samsung","Model":"SM-G970U1","Hardware":"qcom","RadioVersion":"G970U1UES3DTD5","OS":"Android","OSVersion":"10","MAC":"NotAvailable","BatteryPercentCharge":"94"},"SDK":{"MistID":"3093b881-b5a8-4c3e-bd44-7464b0712a5d","SDKName":"Marvis-Location-SDK 1.0","SDKVersion":"2.0.161","App":"Wakeup","AppVersion":"1.0(2)"},"ClientInfoTimestamp":"2020-05-11T19:46:25.0640Z"},"Stats":{"Passive":{"Tests":[{"ConnectionType":"WiFi","CellularData":{"Type":"WCDMA","AsuLevel":4,"RSSI":-105,"Level":1,"CellCID":151072024,"CellOperatorNumber":310260,"CellOperatorNameShort":"T-Mobile"},"WiFiConnectionData":{"BSSID":"5c:5b:35:03:d8:71","SignalStrength":99,"LinkSpeed":450,"Frequency":5660,"RSSI":-55,"SSID":"Marvis","NetworkId":11,"SupplicantState":"COMPLETED"},"WiFiScanData":[{"BSSID":"5c:5b:

In [5]:
df= rdd.map(lambda x: json.loads(x[1])).toDF()

/opt/mistsys/ds_incubator/venv/spark-2.4.4-bin-without-hadoop/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


In [6]:
df.printSchema()

root
 |-- HashKey: string (nullable = true)
 |-- InfoFromTerminator: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- MarvisDataTimestamp: string (nullable = true)
 |-- ReplyPath: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- Result: map (nullable = true)
 |    |-- key: string
 |    |-- value: map (valueContainsNull = true)
 |    |    |-- key: string
 |    |    |-- value: map (valueContainsNull = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |-- Topic: string (nullable = true)
 |-- Type: string (nullable = true)



In [7]:
df.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             HashKey|  InfoFromTerminator| MarvisDataTimestamp|           ReplyPath|              Result|               Topic|                Type|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|3093b881-b5a8-4c3...|[OrgID -> 9c3e516...|2020-05-11T19:46:...|[Topic -> client-...|[Stats -> [Passiv...|mobile-client-mar...|Test send Basic C...|
|3093b881-b5a8-4c3...|[OrgID -> 9c3e516...|2020-05-11T19:46:...|[Topic -> client-...|[Stats -> [Passiv...|mobile-client-mar...|Test send Basic C...|
|3093b881-b5a8-4c3...|[OrgID -> 9c3e516...|2020-05-11T19:46:...|[Topic -> client-...|[Stats -> [Passiv...|mobile-client-mar...|Test send Basic C...|
|3093b881-b5a8-4c3...|[OrgID -> 9c3e516...|2020-05-11T19:47:...|[Topic -> client-...|[Stats -> [Passiv...|

In [8]:
data= rdd.map(lambda x: json.loads(x[1])).first()
import pprint

pprint.pprint(data)


{'HashKey': '3093b881-b5a8-4c3e-bd44-7464b0712a5d',
 'InfoFromTerminator': {'ID': '9c3e516c-397d-11e6-ae35-0242ac110008.3093b881b5a84c3ebd447464b0712a5d',
                        'OrgID': '9c3e516c-397d-11e6-ae35-0242ac110008',
                        'RemoteAddr': '71.204.156.184:47498',
                        'SiteID': None,
                        'Terminator': 'client-terminator-001-staging',
                        'Timestamp': '2020-05-11T19:46:34.494924604Z'},
 'MarvisDataTimestamp': '2020-05-11T19:46:35.1770Z',
 'ReplyPath': {'Opaque': 'AAAdgLZyjNb0W2YtN39ve1A7H+lAcmdmIOMuIvQLr06vFX1qIH7SpwJe80sev8mISogny+OGDLQFBwSq+M6IkeeLtsFhz2NGouNTg0ms7psMmxyTEjstFwhrhSBpVgdphzzBQ7RZe/k075wPRDr1xtdD+AGmVzM8NISn/UMkumfI8lVcF0V0eI0n597mJeyLMePZTJfV6GimoXUzUA6bJx4tCNDl9J0o+pz6hVEC',
               'Topic': 'client-reply-client-terminator-001-staging'},
 'Result': {'ClientInformation': {'ClientInfoTimestamp': '2020-05-11T19:46:25.0640Z',
                                  'Device': {'BatteryPer

In [9]:
df= rdd.map(lambda x: json.loads(x[1])).toDF()


In [10]:
df_device= df.select(explode("Result.ClientInformation.Device"))

In [11]:
df_device.show()

+--------------------+--------------+
|                 key|         value|
+--------------------+--------------+
|BatteryPercentCharge|            94|
|                  OS|       Android|
|               Model|     SM-G970U1|
|        Manufacturer|       Samsung|
|           OSVersion|            10|
|            Hardware|          qcom|
|        RadioVersion|G970U1UES3DTD5|
|                 MAC|  NotAvailable|
|BatteryPercentCharge|            94|
|                  OS|       Android|
|               Model|     SM-G970U1|
|        Manufacturer|       Samsung|
|           OSVersion|            10|
|            Hardware|          qcom|
|        RadioVersion|G970U1UES3DTD5|
|                 MAC|  NotAvailable|
|BatteryPercentCharge|            94|
|                  OS|       Android|
|               Model|     SM-G970U1|
|        Manufacturer|       Samsung|
+--------------------+--------------+
only showing top 20 rows



In [12]:
df.select("Result.ClientInformation.Device").first()

Row(Device={'BatteryPercentCharge': '94', 'OS': 'Android', 'Model': 'SM-G970U1', 'Manufacturer': 'Samsung', 'OSVersion': '10', 'Hardware': 'qcom', 'RadioVersion': 'G970U1UES3DTD5', 'MAC': 'NotAvailable'})

In [13]:

df_device = df.select("HashKey",
                      "InfoFromTerminator.OrgID",
                      "MarvisDataTimestamp",
                      "Result.ClientInformation.Device.MAC",
                 "Result.ClientInformation.Device.Model",
                 "Result.ClientInformation.Device.OS",
                 "Result.ClientInformation.Device.OSVersion",
                 "Result.ClientInformation.Device.Hardware",
                 "Result.ClientInformation.Device.RadioVersion",
                 "Result.ClientInformation.Device.Manufacturer" 
                )

In [14]:
df_device.show()

+--------------------+--------------------+--------------------+------------+---------+-------+---------+--------+--------------+------------+
|             HashKey|               OrgID| MarvisDataTimestamp|         MAC|    Model|     OS|OSVersion|Hardware|  RadioVersion|Manufacturer|
+--------------------+--------------------+--------------------+------------+---------+-------+---------+--------+--------------+------------+
|3093b881-b5a8-4c3...|9c3e516c-397d-11e...|2020-05-11T19:46:...|NotAvailable|SM-G970U1|Android|       10|    qcom|G970U1UES3DTD5|     Samsung|
|3093b881-b5a8-4c3...|9c3e516c-397d-11e...|2020-05-11T19:46:...|NotAvailable|SM-G970U1|Android|       10|    qcom|G970U1UES3DTD5|     Samsung|
|3093b881-b5a8-4c3...|9c3e516c-397d-11e...|2020-05-11T19:46:...|NotAvailable|SM-G970U1|Android|       10|    qcom|G970U1UES3DTD5|     Samsung|
|3093b881-b5a8-4c3...|9c3e516c-397d-11e...|2020-05-11T19:47:...|NotAvailable|SM-G970U1|Android|       10|    qcom|G970U1UES3DTD5|     Samsung|

In [15]:
df_device.groupBy("MAC", "OrgID").count().show(10, False)

+-----------------+------------------------------------+-----+
|MAC              |OrgID                               |count|
+-----------------+------------------------------------+-----+
|2c-0e-3d-f8-ad-f9|eec9929e-4a8a-11e5-b454-1258369c38a9|574  |
|d4-63-c6-eb-1f-a2|b578ad1b-7f7a-43f5-92a9-19b05f983d54|2079 |
|NotAvailable     |eec9929e-4a8a-11e5-b454-1258369c38a9|13   |
|NotAvailable     |b578ad1b-7f7a-43f5-92a9-19b05f983d54|870  |
|NotAvailable     |9c3e516c-397d-11e6-ae35-0242ac110008|89   |
|5c-41-5a-ba-e5-f8|eec9929e-4a8a-11e5-b454-1258369c38a9|383  |
+-----------------+------------------------------------+-----+



In [16]:
df_device.groupBy("MAC",  "Model","OS","OSVersion","Hardware", "RadioVersion", "Manufacturer").count().show()

+-----------------+---------------+-------+---------+----------+--------------------+------------+-----+
|              MAC|          Model|     OS|OSVersion|  Hardware|        RadioVersion|Manufacturer|count|
+-----------------+---------------+-------+---------+----------+--------------------+------------+-----+
|     NotAvailable|         KFMUWI|Android|    7.1.2|    mt8163|                    |      Amazon|   13|
|     NotAvailable|       SM-G970F|Android|       10|exynos9820|G970FXXU3BSKL,G97...|     Samsung|  642|
|2c-0e-3d-f8-ad-f9|       SM-G930U|Android|    8.0.0|      qcom|       G930UUESBCTA3|     Samsung|  574|
|d4-63-c6-eb-1f-a2|Moto G (5) Plus|Android|    8.1.0|      qcom|M8953_41.50.07.85...|    Motorola| 2079|
|     NotAvailable|      SM-G970U1|Android|       10|      qcom|      G970U1UES3DTD5|     Samsung|   89|
|     NotAvailable|       SM-G970F|Android|        9|exynos9820|G970FXXS3ASJA,G97...|     Samsung|  228|
|5c-41-5a-ba-e5-f8|         KFMUWI|Android|    7.1.2|  

In [17]:
df_device.groupBy("MAC", "HashKey").count().orderBy("HashKey").show(50, False)

+-----------------+------------------------------------+-----+
|MAC              |HashKey                             |count|
+-----------------+------------------------------------+-----+
|NotAvailable     |3093b881-b5a8-4c3e-bd44-7464b0712a5d|89   |
|2c-0e-3d-f8-ad-f9|453e5b99-ea8f-4518-822b-7627b7d34c84|574  |
|NotAvailable     |861d31dd-4e19-42a3-8ba4-e1325fe13e5d|13   |
|5c-41-5a-ba-e5-f8|861d31dd-4e19-42a3-8ba4-e1325fe13e5d|383  |
|d4-63-c6-eb-1f-a2|b385e302-a9ee-4c5e-b38b-d8f67c96fab1|2079 |
|NotAvailable     |e041c687-8689-4474-83f4-ee941034ab04|870  |
+-----------------+------------------------------------+-----+



In [18]:
df_c1 = df.filter(col("Result.ClientInformation.Device.MAC")=="2c-0e-3d-f8-ad-f9").show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             HashKey|  InfoFromTerminator| MarvisDataTimestamp|           ReplyPath|              Result|               Topic|                Type|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|453e5b99-ea8f-451...|[OrgID -> eec9929...|2020-05-12T19:40:...|[TCP -> 172.31.56...|[Stats -> [Passiv...|mobile-client-mar...|Test send Basic C...|
|453e5b99-ea8f-451...|[OrgID -> eec9929...|2020-05-12T19:40:...|[TCP -> 172.31.56...|[Stats -> [Passiv...|mobile-client-mar...|Test send Basic C...|
|453e5b99-ea8f-451...|[OrgID -> eec9929...|2020-05-12T19:40:...|[TCP -> 172.31.56...|[Stats -> [Passiv...|mobile-client-mar...|Test send Basic C...|
|453e5b99-ea8f-451...|[OrgID -> eec9929...|2020-05-12T19:40:...|[TCP -> 172.31.56...|[Stats -> [Passiv...|